In [ ]:
# The purpose of this code is to develop a GIF with a map of Minnesota (MN) successively adding MN state highways-just for fun

In [142]:
pip install imageio

     |████████████████████████████████| 3.3 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [143]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import imageio # to make the GIF file

In [5]:
roads = 'STREETS_LOAD.shp'

In [80]:
states = '/Users/matthewwilliamhaas/Documents/shapefile_data_sources/cb_2018_us_county_20m/cb_2018_us_county_20m.shp'
map_states = gpd.read_file(states) # read in counties

In [83]:
MN = map_states[(map_states.STATEFP == '27')]
MN = MN.to_crs({'init' : 'epsg:26915'})

In [6]:
# read in data
road_data = gpd.read_file(roads)

In [183]:
# subset data so we can map each highway independently
MN61 = road_data.STREET_NAM == 'MNTH 61'
MN36 = road_data.STREET_NAM == 'MNTH 36'
MN120 = road_data.STREET_NAM == 'MNTH 120'
MN280 = road_data.STREET_NAM == 'MNTH 280'

In [184]:
# needed to plot positions of highways
MN61_data = road_data[(MN61)].geometry
MN36_data = road_data[(MN36)].geometry
MN120_data = road_data[(MN120)].geometry
MN280_data = road_data[(MN280)].geometry

In [164]:
road_data[(MN280)]

,STREET_NAM,STREET_PRE,BASE_NAME,STREET_TYP,STREET_SUF,E_911,ROUTE_SYST,ROUTE_NUMB,TIS_CODE,DIVIDED_ON,TRAFFIC_DI,SHAPE_Leng,geometry
6306,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,565.695996,"LINESTRING (483915.876 4981782.748, 483866.013..."
6403,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,764.654286,"LINESTRING (483750.340 4982975.841, 483746.245..."
37938,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,189.294034,"LINESTRING (484100.094 4984125.000, 484043.563..."
40741,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,1026.468621,"LINESTRING (483940.625 4984029.888, 483909.872..."
40857,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,440.110534,"LINESTRING (483898.473 4978496.745, 483945.937..."
41691,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,193.678763,"LINESTRING (483954.751 4981171.896, 483955.062..."
104919,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,373.467493,"LINESTRING (483774.303 4982213.099, 483784.394..."
107875,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,150.525192,"LINESTRING (484003.531 4980385.999, 484005.281..."
115869,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,816.532840,"LINESTRING (483930.719 4981197.500, 483930.844..."
166658,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,67.053405,"LINESTRING (483760.190 4983042.158, 483750.678..."


In [69]:
# used this code for the map of MN and WI--not sure if it will be useful here or not
#TH61 = TH61.to_crs({"init" : "epsg:26915"})

In [280]:
# Convert code for making one-off highway maps to a function to do in a high-throughput manner.
# Needed arguments: 1) path to image (highway sign), 2) indicator of current highway to plot geodata, and 3) output filename

# Initiate empty list to hold highways that have already been plotted
#alreadyPlotted = [] # Keep outside function definitation so it doesn't get cleared every time the function is called
#geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

def MakeHwyMap (imagefile, currentHwy, outfile):
    global geoDataAll
    data = road_data.STREET_NAM == currentHwy
    geoData = road_data[(data)].geometry
    im = plt.imread(imagefile)
    fig, ax = plt.subplots(1, figsize = (20,15))
    MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
    #if currentHwy in alreadyPlotted:
    geoDataAll.plot(color ='grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #else:
    geoData.plot(color ='red', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #alreadyPlotted.append(currentHwy)
    geoDataAll = geoDataAll.append(geoData)
    newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
    newax.imshow(im)
    ax.axis('off') # turn off the axis
    newax.axis('off')
    plt.savefig(outfile)
    plt.close('all')

In [283]:
infile = open('minnesota_highway_list.csv', 'r', encoding='utf-8-sig') # open file
highways = infile.readlines()  # begin reading data

out_PNG_list = [] # make empty list to hold names of output files to make GIF at end
#alreadyPlotted = []
geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

# Run the MakeHwyMap function for each line in the file (each line represents a highway)
for line in highways:
    line = line.rstrip()
    line = line.split(',')
    print(line[0])
    MakeHwyMap(imagefile = line[0], currentHwy = line[1], outfile = line[2])
    out_PNG_list.append(line[2])

/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-1.png


/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-3.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-4.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-5.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-6.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-7.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-9.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-11.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-13.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-15.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-16.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-18.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-19.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-20.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-21.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-22.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-23.pn

/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-40.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-41.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-42.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-43.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-44.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-45.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-46.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-47.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-48.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-50.png


/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-51.png


/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-54.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-55.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-56.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-57.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-58.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-60.png


ValueError: invalid PNG header

In [284]:
# make the GIF file
with imageio.get_writer('movie.gif', mode = 'I', duration = 1) as writer:
    for filename in out_PNG_list:
        image = imageio.imread(filename)
        writer.append_data(image)

In [194]:
import os

In [195]:
os.getcwd()

'/Users/matthewwilliamhaas/Documents/shp_trans_roads_mndot_tis'

In [211]:
type(MN61_data)

geopandas.geoseries.GeoSeries

In [279]:
geoDataAll

1759      LINESTRING (509180.419 4986942.865, 507566.813...
3019      LINESTRING (487161.719 4984121.000, 486909.657...
6686      LINESTRING (506733.789 4986923.421, 506363.968...
12535     LINESTRING (499208.552 4984317.019, 499124.250...
13519     LINESTRING (496754.890 4984243.009, 496875.996...
                                ...                        
364342    LINESTRING (484329.031 4984114.999, 484100.094...
396625    LINESTRING (484018.803 4980236.266, 484022.374...
411648    LINESTRING (484032.657 4979631.000, 484050.559...
432362    LINESTRING (483786.556 4982331.808, 483780.206...
433187    LINESTRING (483934.842 4981361.687, 483930.688...
Length: 338, dtype: geometry

In [240]:
MN61_data

1473      LINESTRING (675235.313 5280064.500, 675340.750...
1521      LINESTRING (608055.999 5216460.500, 608088.813...
2530      LINESTRING (670168.533 5277300.863, 670283.876...
3604      LINESTRING (612217.500 5218407.000, 613166.874...
8216      LINESTRING (598735.154 5208051.180, 598782.249...
                                ...                        
435609    LINESTRING (594486.738 5202878.317, 595055.062...
438787    LINESTRING (695409.967 5290808.461, 695457.500...
439156    LINESTRING (635910.248 5244193.280, 635972.699...
439707    LINESTRING (722102.692 5300819.234, 723411.563...
442349    LINESTRING (630215.896 5236859.159, 630338.438...
Name: geometry, Length: 189, dtype: geometry

In [268]:
alreadyPlotted = []

In [269]:
alreadyPlotted

[]